In [ ]:
!pip install -r requirements.txt

In [ ]:
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from uuid import uuid4
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

In [2]:
app = FastAPI()

# Load environment variables from .env file
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)
print(llm)

In [ ]:
# Initialize Neo4j graph connection
graph = Neo4jGraph(url=NEO4J_URL, username=NEO4J_USERNAME,
                   password=NEO4J_PASSWORD)

print(graph)

In [ ]:
# Generate a unique session ID
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

In [9]:
def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

In [37]:
# Define the chat prompt template
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI tutor for Singaporean students, knowledgeable about the Singapore curriculum and 21st-century skills.\n"
                   "Your goal is to provide curriculum-aligned learning experiences while promoting critical thinking, creativity, and cyber wellness.\n"
                   "Current subject: {subject}\n"
                   "Current topic: {topic}\n\n"
                   "Please provide responses that:\n"
                   "1. Are accurate and aligned with the Singapore curriculum\n"
                   "2. Encourage critical thinking and problem-solving\n"
                   "3. Foster creativity and innovation\n"
                   "4. Promote digital literacy and cyber wellness\n"
                   "5. Are appropriate for the student's education level: {educationLevel}\n\n"
                   "Remember to be encouraging and supportive in your responses."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [38]:
# Create the chat chain with message history
chat_chain = chat_prompt | llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)

print(chat_with_message_history)

class ChatRequest(BaseModel):
    question: str
    subject: str
    topic: str
    educationLevel: str

@app.post("/chat/")
def chat(request: ChatRequest):
    try:
        response = chat_with_message_history.run(
            question=request.question,
            context=request.context,
            subject=request.subject,
            topic=request.topic,
            educationLevel=request.educationLevel,
        )
        return response
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
    
@app.get("/")
def read_root():
    return {"Hello": "World"}


bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]) kwargs={} config={} config_factories=[] get_session_history=<function get_memory at 0x1323c28e0> input_messages_key='question' history_messages_key='chat_history' history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)]


In [46]:
# Invoke the chat chain with a sample request
sample_request = ChatRequest(
    question="What does carbon dioxide do to the environment?",
    subject="Science",
    topic="Chemistry",
    educationLevel="Primary",
    context=""
)

response = chat_with_message_history.invoke(
    {"question": sample_request.question, "subject": sample_request.subject, "topic": sample_request.topic,
        "educationLevel": sample_request.educationLevel, "context": sample_request.context},
    {"configurable": {"session_id": SESSION_ID}}
)

print(response)

Carbon dioxide is a greenhouse gas that contributes to global warming and climate change. When there is an excess of carbon dioxide in the atmosphere, it traps heat from the sun, leading to the Earth's temperature rising. This phenomenon is known as the greenhouse effect.

Excessive carbon dioxide emissions are primarily caused by human activities such as burning fossil fuels (coal, oil, and natural gas), deforestation, and industrial processes. These activities increase the concentration of carbon dioxide in the atmosphere, which disrupts the Earth's climate system.

Can you think of ways we can reduce carbon dioxide emissions to help mitigate the effects of climate change? How can individuals contribute to reducing carbon dioxide levels in the environment?
